In [1]:
import os, sys, email
import numpy as np
import pandas as pd
from nltk import tokenize
import pprint, re, time

pd.options.display.max_colwidth = 1000

In [2]:
from loadFile import getPathList, fileToDataFrame

# ディレクトリ 内のメールファイルを読み込む
directory_path = "enron/maildir/allen-p/inbox"
file_list, dir_list=getPathList(directory_path)

# テキストファイルをデータフレームに格納する
df = fileToDataFrame(file_list)

In [3]:
#ドキュメント（メッセージ）をセンテンスに分解する
from preProcess import toSentences

sent_dict = {'Message-ID':[],
            'Sentence-ID':[],
            'Sentence':[]}

start = time.time()

for i,message in enumerate(df['Message']):
    sentences = toSentences(message)
    sent_dict['Message-ID'].extend([df['Message-ID'][i]] * len(sentences))
    sent_dict['Sentence-ID'].extend(range(len(sentences)))
    sent_dict['Sentence'].extend(sentences)

sentence_df = pd.DataFrame.from_dict(sent_dict,orient='columns')

end = time.time()
print(end-start,'s')

0.10651493072509766 s


In [4]:
from extraction import tripleExtraction, entityExtraction, NERtripleExtraction

triple_dict = {'Message-ID':[],
             'Sentence-ID':[],
             'Triple':[]}

start = time.time()

for i, sentence in enumerate(sentence_df['Sentence']):
    triples = tripleExtraction(sentence)
    entities,entities_type = entityExtraction(sentence)
    nertriples = NERtripleExtraction(triples, entities)
    if nertriples is None:
        pass
    else:
        triple_dict['Message-ID'].extend([sentence_df['Message-ID'][i]] * len(nertriples))
        triple_dict['Sentence-ID'].extend([sentence_df['Sentence-ID'][i]] * len(nertriples))
        triple_dict['Triple'].extend(nertriples)

triple_df = pd.DataFrame.from_dict(triple_dict,orient='columns')

end = time.time()
print(end-start,'s')

214.54530596733093 s


In [6]:
output = pd.merge(df[['Path','Message-ID','Subject']],pd.merge(sentence_df,triple_df,on=['Message-ID','Sentence-ID']),on='Message-ID')
output.to_csv('output/allen_p_inbox_triple4.csv')

#output = pd.read_csv('output/allen_p_inbox_triple.csv')

In [28]:
output.groupby(['Path','Sentence-ID','Sentence','Triple']).agg({'Triple':'unique'}).to_csv('output/allen_p_inbox_triple4.csv')